# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-05-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-05-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-06 04:33:57,34.223334,-82.461707,124,0,0,124.0,"Abbeville, South Carolina, US",505.565295,0.000000
1,22001.0,Acadia,Louisiana,US,2020-07-06 04:33:57,30.295065,-92.414197,1016,40,0,976.0,"Acadia, Louisiana, US",1637.521154,3.937008
2,51001.0,Accomack,Virginia,US,2020-07-06 04:33:57,37.767072,-75.632346,1044,14,0,1030.0,"Accomack, Virginia, US",3230.597846,1.340996
3,16001.0,Ada,Idaho,US,2020-07-06 04:33:57,43.452658,-116.241552,2835,23,0,2812.0,"Ada, Idaho, US",588.678681,0.811287
4,19001.0,Adair,Iowa,US,2020-07-06 04:33:57,41.330756,-94.471059,15,0,0,15.0,"Adair, Iowa, US",209.731544,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,30451,30616,30967,31238,31517,31836,32022,32324,32672,32951
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2269,2330,2402,2466,2535,2580,2662,2752,2819,2893
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,12685,12968,13273,13571,13907,14272,14657,15070,15500,15941


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,683,703,721,733,746,774,807,819,826,864
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,51,53,55,58,62,65,69,72,74,76
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,885,892,897,905,912,920,928,937,946,952


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,10306,10674,12604,13934,14131,15651,16041,17331,19164,19366
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1298,1346,1384,1438,1459,1516,1559,1592,1637,1657
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,9066,9202,9371,9674,9897,10040,10342,10832,11181,11492


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,247,247,247,253,257,267,280,280,280,280
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,30,30,30,30,30,31,31,31,31


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1951,1103.0,Morgan,Alabama,US,2020-07-06 04:33:57,34.455006,-86.854759,1102,5,0,1097.0,"Morgan, Alabama, US",920.796464,0.453721
1529,1075.0,Lamar,Alabama,US,2020-07-06 04:33:57,33.779950,-88.096680,77,1,0,76.0,"Lamar, Alabama, US",557.768924,1.298701
1553,1077.0,Lauderdale,Alabama,US,2020-07-06 04:33:57,34.901719,-87.656247,490,6,0,484.0,"Lauderdale, Alabama, US",528.421529,1.224490


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2888635,129947,906763
Brazil,1603055,64867,1029045
India,697413,19693,424433


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2888635,129947,906763,1851925,2020-07-06 04:33:57,38.304583,-91.659256
Brazil,1603055,64867,1029045,509143,2020-07-06 04:33:57,-12.669522,-48.480493
India,697413,19693,424433,253287,2020-07-06 04:33:57,23.356511,82.081064
Russia,680283,10145,449995,220143,2020-07-06 04:33:57,54.546312,62.120860
Peru,302718,10589,193957,98172,2020-07-06 04:33:57,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,397131,32206,0
California,264681,6373,0
Florida,200111,3731,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,397131,32206,0,364925,2020-07-06 04:33:57,42.671593,-75.579694
California,264681,6373,0,258308,2020-07-06 04:33:57,37.778248,-120.728666
Florida,200111,3731,0,196380,2020-07-06 04:33:57,28.940755,-82.700744
Texas,194932,2628,0,192304,2020-07-06 04:33:57,31.639310,-98.596074
New Jersey,173402,15211,0,158191,2020-07-06 04:33:57,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,216969,23190,0
California,Los Angeles,115024,3487,0
Illinois,Cook,92532,4627,0
Arizona,Maricopa,62296,883,0
Florida,Miami-Dade,47011,1043,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,216969,23190,0,193779,2020-07-06 04:33:57,40.767273,-73.971526,36061.0
California,Los Angeles,115024,3487,0,111537,2020-07-06 04:33:57,34.308284,-118.228241,6037.0
Illinois,Cook,92532,4627,0,87905,2020-07-06 04:33:57,41.841448,-87.816588,17031.0
Arizona,Maricopa,62296,883,0,61413,2020-07-06 04:33:57,33.348359,-112.491815,4013.0
Florida,Miami-Dade,47011,1043,0,45968,2020-07-06 04:33:57,25.611236,-80.551706,12086.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,32324,2752,15070,855,328,68,72786,27320,8260,18050,...,285787,952,9396,6537,355,3334,10,1240,1632,625
2020-07-04,32672,2819,15500,855,346,68,75376,27900,8443,18165,...,286412,955,9708,6750,355,3835,10,1248,1632,698
2020-07-05,32951,2893,15941,855,346,68,77815,28606,8583,18280,...,286931,956,10020,7169,355,4277,10,1265,1632,716


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,819,72,937,52,18,3,1437,469,104,705,...,44216,28,29,59,0,11,1,335,30,7
2020-07-04,826,74,946,52,19,3,1481,477,104,705,...,44283,28,31,62,0,13,1,337,30,8
2020-07-05,864,76,952,52,19,3,1507,484,106,706,...,44305,28,34,65,0,16,1,338,30,8


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,17331,1592,10832,800,107,23,25224,15484,7319,16558,...,1375,837,6251,2100,340,463,8,536,1348,176
2020-07-04,19164,1637,11181,800,108,23,25930,15935,7399,16607,...,1375,840,6425,2100,340,463,8,537,1348,181
2020-07-05,19366,1657,11492,800,108,23,27597,16140,7420,16615,...,1375,849,6584,2100,340,491,8,552,1348,181


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,1326,1389,1408,1453,1495,1537,1622,1680,1702,1711
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,184,188,189,197,200,203,206,214,219,224
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,260,264,266,273,277,286,299,300,302,302
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,159,160,161,165,166,170,177,181,185,194


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-03,41865,1060,0,91872,22622,250514,33595,46717,11923,49,...,48712,185591,23866,1236,98,64393,34778,3126,30317,1582
2020-07-04,42862,1107,0,94567,23209,252895,33849,46717,11996,49,...,50140,192153,24542,1238,111,65109,35247,3205,31055,1606
2020-07-05,43953,1134,0,98103,23814,264681,34048,46717,12128,49,...,51316,194932,24952,1249,111,65748,35898,3262,31577,1634


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-03         560     828     345  186    226     367    623     322   
2020-07-04         583     846     347  187    230     368    624     332   
2020-07-05         607     864     349  190    235     369    633     354   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-03          612       85  ...      61      4       27        6   
2020-07-04          624       85  ...      61      4       27        6   
2020-07-05          629       97  ...      62      4       27        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-03            100   135   182          0       39      2  
2020-07-04            102   137   183          0       39      2  
2020-07-05            106   140   184          0       39      2  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-03,1006,15,0,1798,281,6315,1701,4335,512,0,...,633,2592,181,56,6,1845,1352,93,796,20
2020-07-04,1007,16,0,1817,286,6334,1701,4335,512,0,...,637,2607,181,56,6,1849,1354,94,796,20
2020-07-05,1007,16,0,1825,287,6373,1701,4335,512,0,...,645,2628,184,56,6,1853,1359,94,796,20


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-03          12       9       2    1      1       9     28       5   
2020-07-04          12       9       2    1      1      10     28       5   
2020-07-05          12       9       2    1      1      10     28       5   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-03           27        7  ...       0      0        0        0   
2020-07-04           27        7  ...       0      0        0        0   
2020-07-05           27        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-03              0     0     0         19        0      0  
2020-07-04              0     0     0         19        0      0  
2020-07-05              0     0     0         19        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,0.009431,0.033809,0.028178,0.0,0.041270,-0.014493,0.040677,0.024833,0.024052,0.006075,...,0.001760,0.005280,0.035030,0.042085,0.0,0.082468,0.0,0.015561,0.0,0.012966
2020-07-04,0.010766,0.024346,0.028534,0.0,0.054878,0.000000,0.035584,0.021230,0.022155,0.006371,...,0.002187,0.003151,0.033206,0.032584,0.0,0.150270,0.0,0.006452,0.0,0.116800
2020-07-05,0.008539,0.026250,0.028452,0.0,0.000000,0.000000,0.032358,0.025305,0.016582,0.006331,...,0.001812,0.001047,0.032138,0.062074,0.0,0.115254,0.0,0.013622,0.0,0.025788


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,0.014870,0.043478,0.009698,0.0,0.058824,0.0,0.037545,0.021786,0.000000,0.000000,...,0.003085,0.0,0.074074,0.035088,NaN,0.222222,0.0,0.030769,0.0,0.000000
2020-07-04,0.008547,0.027778,0.009605,0.0,0.055556,0.0,0.030619,0.017058,0.000000,0.000000,...,0.001515,0.0,0.068966,0.050847,NaN,0.181818,0.0,0.005970,0.0,0.142857
2020-07-05,0.046005,0.027027,0.006342,0.0,0.000000,0.0,0.017556,0.014675,0.019231,0.001418,...,0.000497,0.0,0.096774,0.048387,NaN,0.230769,0.0,0.002967,0.0,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,0.080419,0.021167,0.047380,0.0,0.103093,0.0,0.042917,0.029795,0.026508,0.002664,...,0.001457,0.010870,0.035963,0.0,0.0,0.006522,0.0,0.044834,0.0,0.017341
2020-07-04,0.105764,0.028266,0.032219,0.0,0.009346,0.0,0.027989,0.029127,0.010930,0.002959,...,0.000000,0.003584,0.027836,0.0,0.0,0.000000,0.0,0.001866,0.0,0.028409
2020-07-05,0.010541,0.012217,0.027815,0.0,0.000000,0.0,0.064288,0.012865,0.002838,0.000482,...,0.000000,0.010714,0.024747,0.0,0.0,0.060475,0.0,0.027933,0.0,0.000000


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-03,0.043729,0.045365,NaN,0.050626,0.024779,0.016078,0.007800,0.001522,0.016367,0.0,...,0.038857,0.036028,0.025612,0.007335,0.065217,0.010324,0.018360,0.023911,0.019470,0.020645
2020-07-04,0.023815,0.044340,NaN,0.029334,0.025948,0.009504,0.007561,0.000000,0.006123,0.0,...,0.029315,0.035357,0.028325,0.001618,0.132653,0.011119,0.013486,0.025272,0.024343,0.015171
2020-07-05,0.025454,0.024390,NaN,0.037391,0.026067,0.046604,0.005879,0.000000,0.011004,0.0,...,0.023454,0.014462,0.016706,0.008885,0.000000,0.009814,0.018470,0.017785,0.016809,0.017435


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-03      0.012658  0.126531  0.036036  0.056818  0.036697  0.008242   
2020-07-04      0.041071  0.021739  0.005797  0.005376  0.017699  0.002725   
2020-07-05      0.041166  0.021277  0.005764  0.016043  0.021739  0.002717   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-03      0.026359  0.150000  0.020000  0.075949  ...  0.070175    0.0   
2020-07-04      0.001605  0.031056  0.019608  0.000000  ...  0.000000    0.0   
2020-07-05      0.014423  0.066265  0.008013  0.141176  ...  0.016393    0.0   

Province_State                                                              \
Admin2         Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-03          0.0      0.0   0.086957 -0.007353  0.011111        NaN   
2020-07-04          0.0      0.0   0.020000  0.014815  0.005495        NaN   
2020-07-05          0.0      0.0   0.039216  0.021898  0.005464        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-07-03          0.0    0.0  
2020-07-04          0.0    0.0  
2020-07-05          0.0    0.0  

[3 rows x 3254 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-03,0.021320,0.071429,NaN,0.019274,0.007168,0.007981,0.0,0.00208,0.003922,NaN,...,0.020968,0.019670,0.028409,0.0,0.0,0.015969,0.007452,0.000000,0.003783,0.0
2020-07-04,0.000994,0.066667,NaN,0.010567,0.017794,0.003009,0.0,0.00000,0.000000,NaN,...,0.006319,0.005787,0.000000,0.0,0.0,0.002168,0.001479,0.010753,0.000000,0.0
2020-07-05,0.000000,0.000000,NaN,0.004403,0.003497,0.006157,0.0,0.00000,0.000000,NaN,...,0.012559,0.008055,0.016575,0.0,0.0,0.002163,0.003693,0.000000,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga Baldwin Barbour Bibb Blount   Bullock    Butler   
2020-07-03         0.0     0.0     1.0  0.0    0.0  0.000000  0.037037   
2020-07-04         0.0     0.0     0.0  0.0    0.0  0.111111  0.000000   
2020-07-05         0.0     0.0     0.0  0.0    0.0  0.000000  0.000000   

Province_State                            ... Wyoming                  \
Admin2         Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-07-03         0.0      0.0      0.0  ...     NaN    NaN      NaN   
2020-07-04         0.0      0.0      0.0  ...     NaN    NaN      NaN   
2020-07-05         0.0      0.0      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-03          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-04          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-05          NaN        NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,0.008583,0.030610,0.027037,2.264854e-08,0.049625,-0.003355,0.040552,0.023266,0.016570,0.005124,...,0.001129,0.005219,0.034344,0.040894,0.000065,0.101086,0.000027,0.020213,0.004626,0.016911
2020-07-04,0.009674,0.027478,0.027785,1.132427e-08,0.052252,-0.001678,0.038068,0.022248,0.019362,0.005748,...,0.001658,0.004185,0.033775,0.036739,0.000032,0.125678,0.000014,0.013332,0.002313,0.066855
2020-07-05,0.009107,0.026864,0.028118,5.662134e-09,0.026126,-0.000839,0.035213,0.023776,0.017972,0.006039,...,0.001735,0.002616,0.032957,0.049406,0.000016,0.120466,0.000007,0.013477,0.001157,0.046322


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,0.025263,0.050387,0.009102,7.479799e-08,0.094940,8.336430e-25,0.032522,0.019633,0.000014,0.000306,...,0.002853,0.005272,0.057890,0.046827,NaN,0.190278,0.0,0.025769,0.038638,0.005269
2020-07-04,0.016905,0.039083,0.009353,3.739899e-08,0.075248,4.168215e-25,0.031571,0.018345,0.000007,0.000153,...,0.002184,0.002636,0.063428,0.048837,NaN,0.186048,0.0,0.015870,0.019319,0.074063
2020-07-05,0.031455,0.033055,0.007848,1.869950e-08,0.037624,2.084107e-25,0.024563,0.016510,0.009619,0.000786,...,0.001341,0.001318,0.080101,0.048612,NaN,0.208409,0.0,0.009418,0.009659,0.037032


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-03,0.067387,0.025664,0.035991,0.000325,0.061675,0.011364,0.046091,0.032809,0.015792,0.002090,...,0.001326,0.006808,0.033536,0.072311,0.003837,0.006371,1.589457e-07,0.040312,0.003683,0.030213
2020-07-04,0.086575,0.026965,0.034105,0.000162,0.035511,0.005682,0.037040,0.030968,0.013361,0.002525,...,0.000663,0.005196,0.030686,0.036155,0.001919,0.003186,7.947286e-08,0.021089,0.001841,0.029311
2020-07-05,0.048558,0.019591,0.030960,0.000081,0.017755,0.002841,0.050664,0.021916,0.008100,0.001503,...,0.000331,0.007955,0.027716,0.018078,0.000959,0.031830,3.973643e-08,0.024511,0.000921,0.014655


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-03,0.035840,0.041897,NaN,0.048775,0.028516,0.024277,0.008404,0.001628,0.014612,5.245086e-34,...,0.037124,0.039983,0.024972,0.007636,0.052210,0.009209,0.018898,0.023087,0.018958,0.021510
2020-07-04,0.029827,0.043118,NaN,0.039054,0.027232,0.016891,0.007982,0.000814,0.010367,2.622543e-34,...,0.033220,0.037670,0.026648,0.004627,0.092431,0.010164,0.016192,0.024180,0.021650,0.018340
2020-07-05,0.027641,0.033754,NaN,0.038223,0.026650,0.031748,0.006931,0.000407,0.010685,1.311271e-34,...,0.028337,0.026066,0.021677,0.006756,0.046216,0.009989,0.017331,0.020982,0.019230,0.017887


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-03      0.016620  0.094522  0.027585  0.042042  0.034006  0.007041   
2020-07-04      0.028846  0.058130  0.016691  0.023709  0.025852  0.004883   
2020-07-05      0.035006  0.039703  0.011227  0.019876  0.023796  0.003800   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-03      0.014968  0.090538  0.021332  0.063198  ...  0.078980   
2020-07-04      0.008286  0.060797  0.020470  0.031599  ...  0.039490   
2020-07-05      0.011355  0.063531  0.014241  0.086388  ...  0.027942   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-03      0.073161  0.004369  0.001629   0.056017  0.010220  0.012715   
2020-07-04      0.036580  0.002185  0.000814   0.038009  0.012517  0.009105   
2020-07-05      0.018290  0.001092  0.000407   0.038612  0.017207  0.007285   

Province_State                                     
Admin2         Unassigned      Washakie    Weston  
2020-07-03            NaN  5.446197e-11  0.031250  
2020-07-04            NaN  2.723099e-11  0.015625  
2020-07-05            NaN  1.361549e-11  0.007813  

[3 rows x 3254 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-03,0.018860,0.036365,NaN,0.023861,0.010731,0.011197,0.001576,0.001315,0.002786,NaN,...,0.017972,0.017698,0.021068,2.774327e-07,3.608282e-17,0.015062,0.005821,0.000103,0.005150,3.394358e-06
2020-07-04,0.009927,0.051516,NaN,0.017214,0.014262,0.007103,0.000788,0.000657,0.001393,NaN,...,0.012145,0.011742,0.010534,1.387163e-07,1.804141e-17,0.008615,0.003650,0.005428,0.002575,1.697179e-06
2020-07-05,0.004964,0.025758,NaN,0.010809,0.008879,0.006630,0.000394,0.000329,0.000697,NaN,...,0.012352,0.009899,0.013554,6.935817e-08,9.020705e-18,0.005389,0.003671,0.002714,0.001288,8.485896e-07


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   
2020-07-03      0.023764  1.430414e-14   0.500  0.0    0.0 -0.003013   
2020-07-04      0.011882  7.152068e-15   0.250  0.0    0.0  0.054049   
2020-07-05      0.005941  3.576034e-15   0.125  0.0    0.0  0.027025   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun      Chambers  Cherokee  ...    Park   
2020-07-03      0.018539  0.000123  1.676896e-07  0.000088  ...     NaN   
2020-07-04      0.009269  0.000062  8.384478e-08  0.000044  ...     NaN   
2020-07-05      0.004635  0.000031  4.192239e-08  0.000022  ...     NaN   

Province_State                                                                \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta    Unassigned   
2020-07-03        NaN      NaN      NaN        NaN   NaN   NaN  3.594039e-06   
2020-07-04        NaN      NaN      NaN        NaN   NaN   NaN  1.797020e-06   
2020-07-05        NaN      NaN      NaN        NaN   NaN   NaN  8.985099e-07   

Province_State                  
Admin2         Washakie Weston  
2020-07-03          NaN    NaN  
2020-07-04          NaN    NaN  
2020-07-05          NaN    NaN  

[3 rows x 3254 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200705,AK,1111,121621.0,NaN,23.0,NaN,NaN,NaN,3.0,...,122732,0,0,60747b92beb9c600e7cb30085266b382691b2c3a,0,0,0,0,0,NaN
1,20200705,AL,43953,405933.0,NaN,885.0,2909.0,NaN,843.0,NaN,...,449886,0,3,261d349a43f69648cc120a21229dcccb34d1f9e2,0,0,0,0,0,NaN
2,20200705,AR,23209,315684.0,NaN,285.0,1536.0,NaN,NaN,70.0,...,338893,0,0,e4312005a93c1c2896dd24d6c6047b2248b86b9b,0,0,0,0,0,NaN
3,20200705,AS,0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,e8ec81459d65d97180f2b6ad7a0c965297866dfb,0,0,0,0,0,NaN
4,20200705,AZ,98089,506273.0,NaN,3182.0,5161.0,821.0,NaN,531.0,...,604362,4,93,96e9663c70796573a0293b89aa4fb74230832854,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200705,AK,1111.0,121621.0,NaN,23.0,NaN,NaN,NaN,3.0,...,122732,0,0,60747b92beb9c600e7cb30085266b382691b2c3a,0,0,0,0,0,NaN
1,20200705,AL,43953.0,405933.0,NaN,885.0,2909.0,NaN,843.0,NaN,...,449886,0,3,261d349a43f69648cc120a21229dcccb34d1f9e2,0,0,0,0,0,NaN
2,20200705,AR,23209.0,315684.0,NaN,285.0,1536.0,NaN,NaN,70.0,...,338893,0,0,e4312005a93c1c2896dd24d6c6047b2248b86b9b,0,0,0,0,0,NaN
3,20200705,AS,0.0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,e8ec81459d65d97180f2b6ad7a0c965297866dfb,0,0,0,0,0,NaN
4,20200705,AZ,98089.0,506273.0,NaN,3182.0,5161.0,821.0,NaN,531.0,...,604362,4,93,96e9663c70796573a0293b89aa4fb74230832854,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-05,AK,1111,121621.0,NaN,23.0,NaN,NaN,NaN,3.0,NaN,...,122732,0,0,60747b92beb9c600e7cb30085266b382691b2c3a,0,0,0,0,0,NaN
2020-07-05,AL,43953,405933.0,NaN,885.0,2909.0,NaN,843.0,NaN,473.0,...,449886,0,3,261d349a43f69648cc120a21229dcccb34d1f9e2,0,0,0,0,0,NaN
2020-07-05,AR,23209,315684.0,NaN,285.0,1536.0,NaN,NaN,70.0,240.0,...,338893,0,0,e4312005a93c1c2896dd24d6c6047b2248b86b9b,0,0,0,0,0,NaN
2020-07-05,AS,0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,e8ec81459d65d97180f2b6ad7a0c965297866dfb,0,0,0,0,0,NaN
2020-07-05,AZ,98089,506273.0,NaN,3182.0,5161.0,821.0,NaN,531.0,NaN,...,604362,4,93,96e9663c70796573a0293b89aa4fb74230832854,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-05,AK,1111.0,121621.0,NaN,23.0,NaN,NaN,NaN,3.0,NaN,...,122732,0,0,60747b92beb9c600e7cb30085266b382691b2c3a,0,0,0,0,0,NaN
2020-07-05,AL,43953.0,405933.0,NaN,885.0,2909.0,NaN,843.0,NaN,473.0,...,449886,0,3,261d349a43f69648cc120a21229dcccb34d1f9e2,0,0,0,0,0,NaN
2020-07-05,AR,23209.0,315684.0,NaN,285.0,1536.0,NaN,NaN,70.0,240.0,...,338893,0,0,e4312005a93c1c2896dd24d6c6047b2248b86b9b,0,0,0,0,0,NaN
2020-07-05,AS,0.0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,e8ec81459d65d97180f2b6ad7a0c965297866dfb,0,0,0,0,0,NaN
2020-07-05,AZ,98089.0,506273.0,NaN,3182.0,5161.0,821.0,NaN,531.0,NaN,...,604362,4,93,96e9663c70796573a0293b89aa4fb74230832854,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE